In [2]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
import pandas as pd
import numpy as np
import seaborn as sns
import torch
from IPython.display import display

current_dir = os.getcwd()
PROJECT_ROOT = os.path.abspath(os.path.join(current_dir, "../../../../../"))

sys.path.append(os.path.join(PROJECT_ROOT, "shared_utilities"))
from metrics import *



In [3]:
ptid_list = [540, 544, 552, 559, 563, 567, 570, 575, 584, 588, 591, 596]

## **Learning Rate = 0.0001**

In [4]:
dir_0001 = os.path.join(PROJECT_ROOT, "models/jpformer/fine_tuning_development_files/learning_rate/0001")

dict_0001 = {}

detailed_results_df_0001 = pd.DataFrame()

for ptid in ptid_list:
    df = pd.read_csv(os.path.join(dir_0001, f"patient_{ptid}/fine_tuning_eval/patient_{ptid}_overall_cg_ega.csv"))
    df.rename(columns={"Unnamed: 0": "Glycaemic Region"}, inplace=True)
    dict_0001[ptid] = df

    detailed_df = pd.read_csv(os.path.join(dir_0001, f"patient_{ptid}/fine_tuning_eval/patient_{ptid}_detailed_test_results.csv"))
    detailed_df['ptid'] = ptid
    detailed_results_df_0001 = pd.concat([detailed_results_df_0001, detailed_df], axis=0)
    detailed_results_df_0001.reset_index(drop=True, inplace=True)
detailed_results_df_0001.head()

,sequence_id,timepoint,true,predicted,dy_true,dy_pred,true_glucose,predicted_glucose,dy_true_glucose,dy_pred_glucose,P_EGA_Class,R_EGA_Class,CG_EGA_Class,glycemic_region,ptid
0,0,0,-0.539494,-0.207795,0.000000,0.000000,115.0,138.30865,0.000000,0.000000,B,A,AP,eu,540
1,0,1,-0.553725,-0.146372,-0.002846,0.012285,114.0,142.62485,-0.199999,0.863238,B,B,AP,eu,540
2,0,2,-0.553725,-0.090998,0.000000,0.011075,114.0,146.51605,0.000000,0.778242,B,A,AP,eu,540
3,0,3,-0.582186,-0.060554,-0.005692,0.006089,112.0,148.65533,-0.400000,0.427855,B,A,AP,eu,540
4,0,4,-0.582186,-0.041587,0.000000,0.003793,112.0,149.98816,0.000000,0.266565,B,A,AP,eu,540


In [5]:
all_0001_dfs = list(dict_0001.values())

In [6]:
all_0001_value_only_dfs = [df.drop(columns=["Glycaemic Region"]) for df in all_0001_dfs]
aggregate_0001_df = sum(all_0001_value_only_dfs)
# add glycaemic region column
aggregate_0001_df["Glycaemic Region"] = all_0001_dfs[0]["Glycaemic Region"]
# move glycaemic region to the front
aggregate_0001_df = aggregate_0001_df[["Glycaemic Region"] + list(aggregate_0001_df.columns[:-1])]

aggregate_0001_df["AP:EP"] = aggregate_0001_df["AP"] / aggregate_0001_df["EP"]

aggregate_0001_df["AP%"] = aggregate_0001_df["AP"] / aggregate_0001_df["Count"] * 100
aggregate_0001_df["BE%"] = aggregate_0001_df["BE"] / aggregate_0001_df["Count"] * 100
aggregate_0001_df["EP%"] = aggregate_0001_df["EP"] / aggregate_0001_df["Count"] * 100

display(aggregate_0001_df)

,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,10539,411,2171,13121,4.854445,80.321622,3.132383,16.545995
1,eu,301812,19611,6215,327638,48.561866,92.117520,5.985569,1.896911
2,hyper,169598,10618,8225,188441,20.619818,90.000584,5.634655,4.364761
3,overall,481949,30640,16611,529200,29.013846,91.071240,5.789872,3.138889


In [7]:
for ptid, df in dict_0001.items():
    df["AP:EP"] = df["AP"] / df["EP"]
    df["AP%"] = df["AP"] / df["Count"] * 100
    df["BE%"] = df["BE"] / df["Count"] * 100
    df["EP%"] = df["EP"] / df["Count"] * 100
    dict_0001[ptid] = df  # Update the dictionary with the modified DataFrame
    print(f"Patient ID: {ptid}")
    display(df)

Patient ID: 540


,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,1968,38,207,2213,9.507246,88.929056,1.717126,9.353818
1,eu,28549,2623,604,31776,47.266556,89.844537,8.254658,1.900806
2,hyper,12397,1296,1110,14803,11.168468,83.746538,8.754982,7.498480
3,overall,42914,3957,1921,48792,22.339407,87.952943,8.109936,3.937121


Patient ID: 544


,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,123,2,35,160,3.514286,76.875000,1.250000,21.875000
1,eu,30758,1151,320,32229,96.118750,95.435788,3.571318,0.992895
2,hyper,15083,657,399,16139,37.802005,93.456844,4.070884,2.472272
3,overall,45964,1810,754,48528,60.960212,94.716452,3.729805,1.553742


Patient ID: 552


...

,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,81,1,14,96,5.785714,84.375000,1.041667,14.583333
1,eu,29307,2003,784,32094,37.381378,91.316134,6.241042,2.442824
2,hyper,25143,1797,966,27906,26.027950,90.098903,6.439475,3.461621
3,overall,54531,3801,1764,60096,30.913265,90.739816,6.324880,2.935304


Patient ID: 591


,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,981,63,1077,2121,0.910864,46.251768,2.970297,50.777935
1,eu,34501,3120,911,38532,37.871570,89.538565,8.097166,2.364269
2,hyper,13919,1039,765,15723,18.194771,88.526363,6.608154,4.865484
3,overall,49401,4222,2753,56376,17.944424,87.627714,7.489002,4.883284


Patient ID: 596


,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,925,12,383,1320,2.415144,70.075758,0.909091,29.015152
1,eu,33782,1656,151,35589,223.721854,94.922588,4.653123,0.424288
2,hyper,12555,745,479,13779,26.210856,91.116917,5.406778,3.476305
3,overall,47262,2413,1013,50688,46.655479,93.241004,4.760496,1.998501


## **Learning Rate = 0.00001**

In [8]:


dir_00001 = os.path.join(PROJECT_ROOT, "models/jpformer/fine_tuning_development_files/learning_rate/00001")

dict_00001 = {}

detailed_results_df_00001 = pd.DataFrame()

for ptid in ptid_list:
    df = pd.read_csv(os.path.join(dir_00001, f"patient_{ptid}/fine_tuning_eval/patient_{ptid}_overall_cg_ega.csv"))
    df.rename(columns={"Unnamed: 0": "Glycaemic Region"}, inplace=True)
    dict_00001[ptid] = df

    detailed_df = pd.read_csv(os.path.join(dir_00001, f"patient_{ptid}/fine_tuning_eval/patient_{ptid}_detailed_test_results.csv"))
    detailed_df['ptid'] = ptid
    detailed_results_df_00001 = pd.concat([detailed_results_df_00001, detailed_df], axis=0)
    detailed_results_df_00001.reset_index(drop=True, inplace=True)
detailed_results_df_00001.head()

,sequence_id,timepoint,true,predicted,dy_true,dy_pred,true_glucose,predicted_glucose,dy_true_glucose,dy_pred_glucose,P_EGA_Class,R_EGA_Class,CG_EGA_Class,glycemic_region,ptid
0,0,0,-0.639109,-0.690754,0.000000,0.000000,108.0,104.37085,0.0,0.000000,A,A,AP,eu,540
1,0,1,-0.624878,-0.718742,0.002846,-0.005597,109.0,102.40416,0.2,-0.393337,A,A,AP,eu,540
2,0,2,-0.624878,-0.753347,0.000000,-0.006921,109.0,99.97241,0.0,-0.486349,A,A,AP,eu,540
3,0,3,-0.596417,-0.763869,0.005692,-0.002104,111.0,99.23305,0.4,-0.147874,A,A,AP,eu,540
4,0,4,-0.596417,-0.782291,0.000000,-0.003684,111.0,97.93855,0.0,-0.258898,A,A,AP,eu,540


In [9]:
all_00001_dfs = list(dict_00001.values())

In [10]:
all_00001_value_only_dfs = [df.drop(columns=["Glycaemic Region"]) for df in all_00001_dfs]
aggregate_00001_df = sum(all_00001_value_only_dfs)
# add glycaemic region column
aggregate_00001_df["Glycaemic Region"] = all_00001_dfs[0]["Glycaemic Region"]
# move glycaemic region to the front
aggregate_00001_df = aggregate_00001_df[["Glycaemic Region"] + list(aggregate_00001_df.columns[:-1])]

aggregate_00001_df["AP:EP"] = aggregate_00001_df["AP"] / aggregate_00001_df["EP"]

aggregate_00001_df["AP%"] = aggregate_00001_df["AP"] / aggregate_00001_df["Count"] * 100
aggregate_00001_df["BE%"] = aggregate_00001_df["BE"] / aggregate_00001_df["Count"] * 100
aggregate_00001_df["EP%"] = aggregate_00001_df["EP"] / aggregate_00001_df["Count"] * 100

display(aggregate_00001_df)

,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,11087,365,1669,13121,6.642900,84.498133,2.781800,12.720067
1,eu,306328,16798,4512,327638,67.891844,93.495870,5.127000,1.377130
2,hyper,172648,9207,6586,188441,26.214394,91.619127,4.885879,3.494993
3,overall,490063,26370,12767,529200,38.385134,92.604497,4.982993,2.412509


In [11]:
for ptid, df in dict_00001.items():
    df["AP:EP"] = df["AP"] / df["EP"]
    df["AP%"] = df["AP"] / df["Count"] * 100
    df["BE%"] = df["BE"] / df["Count"] * 100
    df["EP%"] = df["EP"] / df["Count"] * 100
    dict_00001[ptid] = df  # Update the dictionary with the modified DataFrame
    print(f"Patient ID: {ptid}")
    display(df)

Patient ID: 540


,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,1946,44,223,2213,8.726457,87.934930,1.988251,10.076819
1,eu,29257,2138,381,31776,76.790026,92.072633,6.728348,1.199018
2,hyper,12876,1021,906,14803,14.211921,86.982368,6.897251,6.120381
3,overall,44079,3203,1510,48792,29.191391,90.340630,6.564601,3.094770


Patient ID: 544


,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,113,2,45,160,2.511111,70.625000,1.250000,28.125000
1,eu,31027,1010,192,32229,161.598958,96.270440,3.133824,0.595737
2,hyper,15405,481,253,16139,60.889328,95.452011,2.980358,1.567631
3,overall,46545,1493,490,48528,94.989796,95.913699,3.076574,1.009726


Patient ID: 552


...

,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,86,3,7,96,12.285714,89.583333,3.125000,7.291667
1,eu,29526,1836,732,32094,40.336066,91.998504,5.720695,2.280800
2,hyper,25616,1510,780,27906,32.841026,91.793879,5.411023,2.795098
3,overall,55228,3349,1519,60096,36.358130,91.899627,5.572750,2.527622


Patient ID: 591


,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,1432,116,573,2121,2.499127,67.515323,5.469118,27.015559
1,eu,35179,2769,584,38532,60.238014,91.298142,7.186235,1.515623
2,hyper,13916,1000,807,15723,17.244114,88.507282,6.360109,5.132608
3,overall,50527,3885,1964,56376,25.726578,89.625018,6.891230,3.483752


Patient ID: 596


,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,991,8,321,1320,3.087227,75.075758,0.606061,24.318182
1,eu,34002,1498,89,35589,382.044944,95.540757,4.209166,0.250077
2,hyper,12567,747,465,13779,27.025806,91.204006,5.421293,3.374701
3,overall,47560,2253,875,50688,54.354286,93.828914,4.444839,1.726247


## **learning rate = 0.000001**

In [12]:
dir_000001 = os.path.join(PROJECT_ROOT, "models/jpformer/fine_tuning_development_files/learning_rate/000001")

dict_000001 = {}

detailed_results_df_000001 = pd.DataFrame()

for ptid in ptid_list:
    df = pd.read_csv(os.path.join(dir_000001, f"patient_{ptid}/fine_tuning_eval/patient_{ptid}_overall_cg_ega.csv"))
    df.rename(columns={"Unnamed: 0": "Glycaemic Region"}, inplace=True)
    dict_000001[ptid] = df

    detailed_df = pd.read_csv(os.path.join(dir_000001, f"patient_{ptid}/fine_tuning_eval/patient_{ptid}_detailed_test_results.csv"))
    detailed_df['ptid'] = ptid
    detailed_results_df_000001 = pd.concat([detailed_results_df_000001, detailed_df], axis=0)
    detailed_results_df_000001.reset_index(drop=True, inplace=True)
detailed_results_df_000001.head()

,sequence_id,timepoint,true,predicted,dy_true,dy_pred,true_glucose,predicted_glucose,dy_true_glucose,dy_pred_glucose,P_EGA_Class,R_EGA_Class,CG_EGA_Class,glycemic_region,ptid
0,0,0,-0.539494,-0.407716,0.000000,0.000000,115.0,124.260130,0.0,0.000000,A,A,AP,eu,540
1,0,1,-0.610648,-0.403161,-0.014231,0.000911,110.0,124.580185,-1.0,0.064010,A,B,AP,eu,540
2,0,2,-0.639109,-0.394774,-0.005692,0.001677,108.0,125.169530,-0.4,0.117870,A,A,AP,eu,540
3,0,3,-0.653340,-0.403660,-0.002846,-0.001777,107.0,124.545120,-0.2,-0.124882,A,A,AP,eu,540
4,0,4,-0.667570,-0.397537,-0.002846,0.001225,106.0,124.975390,-0.2,0.086054,A,A,AP,eu,540


In [13]:
all_000001_dfs = list(dict_000001.values())

In [14]:
all_000001_value_only_dfs = [df.drop(columns=["Glycaemic Region"]) for df in all_000001_dfs]
aggregate_000001_df = sum(all_000001_value_only_dfs)
# add glycaemic region column
aggregate_000001_df["Glycaemic Region"] = all_000001_dfs[0]["Glycaemic Region"]
# move glycaemic region to the front
aggregate_000001_df = aggregate_000001_df[["Glycaemic Region"] + list(aggregate_000001_df.columns[:-1])]

aggregate_000001_df["AP:EP"] = aggregate_000001_df["AP"] / aggregate_000001_df["EP"]

aggregate_000001_df["AP%"] = aggregate_000001_df["AP"] / aggregate_000001_df["Count"] * 100
aggregate_000001_df["BE%"] = aggregate_000001_df["BE"] / aggregate_000001_df["Count"] * 100
aggregate_000001_df["EP%"] = aggregate_000001_df["EP"] / aggregate_000001_df["Count"] * 100

display(aggregate_000001_df)

,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,11173,379,1569,13121,7.121096,85.153571,2.888499,11.957930
1,eu,307104,16670,3864,327638,79.478261,93.732717,5.087932,1.179350
2,hyper,172705,9181,6555,188441,26.347063,91.649376,4.872082,3.478542
3,overall,490982,26230,11988,529200,40.956123,92.778156,4.956538,2.265306


In [15]:
for ptid, df in dict_000001.items():
    df["AP:EP"] = df["AP"] / df["EP"]
    df["AP%"] = df["AP"] / df["Count"] * 100
    df["BE%"] = df["BE"] / df["Count"] * 100
    df["EP%"] = df["EP"] / df["Count"] * 100
    dict_000001[ptid] = df  # Update the dictionary with the modified DataFrame
    print(f"Patient ID: {ptid}")
    display(df)

Patient ID: 540


,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,1916,50,247,2213,7.757085,86.579304,2.259376,11.161319
1,eu,29397,2128,251,31776,117.119522,92.513218,6.696878,0.789904
2,hyper,12948,971,884,14803,14.647059,87.468756,6.559481,5.971762
3,overall,44261,3149,1382,48792,32.026773,90.713642,6.453927,2.832432


Patient ID: 544


,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,127,0,33,160,3.848485,79.375000,0.000000,20.625000
1,eu,31041,1044,144,32229,215.562500,96.313879,3.239319,0.446803
2,hyper,15237,606,296,16139,51.476351,94.411054,3.754879,1.834067
3,overall,46405,1650,473,48528,98.107822,95.625206,3.400099,0.974695


Patient ID: 552


...

,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,88,7,1,96,88.000000,91.666667,7.291667,1.041667
1,eu,29570,1912,612,32094,48.316993,92.135602,5.957500,1.906898
2,hyper,25810,1477,619,27906,41.696284,92.489070,5.292769,2.218161
3,overall,55468,3396,1232,60096,45.022727,92.298988,5.650958,2.050053


Patient ID: 591


,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,1587,74,460,2121,3.450000,74.823197,3.488920,21.687883
1,eu,35311,2700,521,38532,67.775432,91.640714,7.007163,1.352123
2,hyper,14147,876,700,15723,20.210000,89.976468,5.571456,4.452077
3,overall,51045,3650,1681,56376,30.365854,90.543848,6.474386,2.981765


Patient ID: 596


,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,1061,5,254,1320,4.177165,80.378788,0.378788,19.242424
1,eu,33964,1506,119,35589,285.411765,95.433982,4.231645,0.334373
2,hyper,12344,820,615,13779,20.071545,89.585601,5.951085,4.463314
3,overall,47369,2331,988,50688,47.944332,93.452099,4.598722,1.949179


## **Learning Rate Comparison**

In [16]:
display(aggregate_0001_df)

display(aggregate_00001_df)

display(aggregate_000001_df)

,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,10539,411,2171,13121,4.854445,80.321622,3.132383,16.545995
1,eu,301812,19611,6215,327638,48.561866,92.117520,5.985569,1.896911
2,hyper,169598,10618,8225,188441,20.619818,90.000584,5.634655,4.364761
3,overall,481949,30640,16611,529200,29.013846,91.071240,5.789872,3.138889


,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,11087,365,1669,13121,6.642900,84.498133,2.781800,12.720067
1,eu,306328,16798,4512,327638,67.891844,93.495870,5.127000,1.377130
2,hyper,172648,9207,6586,188441,26.214394,91.619127,4.885879,3.494993
3,overall,490063,26370,12767,529200,38.385134,92.604497,4.982993,2.412509


,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,11173,379,1569,13121,7.121096,85.153571,2.888499,11.957930
1,eu,307104,16670,3864,327638,79.478261,93.732717,5.087932,1.179350
2,hyper,172705,9181,6555,188441,26.347063,91.649376,4.872082,3.478542
3,overall,490982,26230,11988,529200,40.956123,92.778156,4.956538,2.265306


In [17]:
results_list = [aggregate_0001_df, aggregate_00001_df, aggregate_000001_df]
results_dict = {}

for i, table in enumerate(results_list):
    #delete last row
    table = table[:-1]
    average_ap_ep = table["AP:EP"].mean()
    results_dict[f"Table {i+1}"] = average_ap_ep

for key, value in results_dict.items():
    print(f"{key}: {value}")

Table 1: 24.67870967918421
Table 2: 33.58304602705514
Table 3: 37.64880680655278


In [18]:
best_hypo_00001 = []
best_hypo_0001 = []

best_hypo_000001 = []

for ptid in ptid_list:

    hypo_dict = {}
    hypo_00001 = dict_00001[ptid].loc[dict_00001[ptid]["Glycaemic Region"] == "hypo"]
    hypo_00001_apep = hypo_00001["AP:EP"].values[0]
    hypo_dict["00001"] = hypo_00001_apep

    hypo_0001 = dict_0001[ptid].loc[dict_0001[ptid]["Glycaemic Region"] == "hypo"]
    hypo_0001_apep = hypo_0001["AP:EP"].values[0]
    hypo_dict["0001"] = hypo_0001_apep

    hypo_000001 = dict_000001[ptid].loc[dict_000001[ptid]["Glycaemic Region"] == "hypo"]
    hypo_000001_apep = hypo_000001["AP:EP"].values[0]
    hypo_dict["000001"] = hypo_000001_apep


    best_hypo = max(hypo_dict, key=hypo_dict.get)
    if best_hypo == "00001":
        best_hypo_00001.append(ptid)
    elif best_hypo == "0001":
        best_hypo_0001.append(ptid)
    elif best_hypo == "000001":
        best_hypo_000001.append(ptid)
    else:
        print("Error: No best hypo found for patient ID:", ptid)

print("Batch size 0001 results in best Hypo AP:EP for ptid: ", best_hypo_0001)
print("Batch size 00001 results in best Hypo AP:EP for ptid: ", best_hypo_00001)
print("Batch size 000001 results in best Hypo AP:EP for ptid: ", best_hypo_000001)



Batch size 0001 results in best Hypo AP:EP for ptid:  [540, 552, 563, 570, 575]
Batch size 00001 results in best Hypo AP:EP for ptid:  [559, 567]
Batch size 000001 results in best Hypo AP:EP for ptid:  [544, 584, 588, 591, 596]


In [19]:
best_overall_00001 = []
best_overall_0001 = []

best_overall_000001 = []

for ptid in ptid_list:

    overall_dict = {}
    overall_00001 = dict_00001[ptid].loc[dict_00001[ptid]["Glycaemic Region"] == "overall"]
    overall_00001_apep = overall_00001["AP:EP"].values[0]
    overall_dict["00001"] = overall_00001_apep

    overall_0001 = dict_0001[ptid].loc[dict_0001[ptid]["Glycaemic Region"] == "overall"]
    overall_0001_apep = overall_0001["AP:EP"].values[0]
    overall_dict["0001"] = overall_0001_apep

    overall_000001 = dict_000001[ptid].loc[dict_000001[ptid]["Glycaemic Region"] == "overall"]
    overall_000001_apep = overall_000001["AP:EP"].values[0]
    overall_dict["000001"] = overall_000001_apep


    best_overall = max(overall_dict, key=overall_dict.get)
    if best_overall == "00001":
        best_overall_00001.append(ptid)
    elif best_overall == "0001":
        best_overall_0001.append(ptid)
    elif best_overall == "000001":
        best_overall_000001.append(ptid)
    else:
        print("Error: No best hypo found for patient ID:", ptid)
print("Batch size 0001 results in best Overall AP:EP for ptid: ", best_overall_0001)
print("Batch size 00001 results in best Overall AP:EP for ptid: ", best_overall_00001)
print("Batch size 000001 results in best Overall AP:EP for ptid: ", best_overall_000001)

Batch size 0001 results in best Overall AP:EP for ptid:  []
Batch size 00001 results in best Overall AP:EP for ptid:  [559, 567, 570, 596]
Batch size 000001 results in best Overall AP:EP for ptid:  [540, 544, 552, 563, 575, 584, 588, 591]
